In [3]:
!pip install langchain HuggingFace chromadb tiktoken pypdf langchain_google_genai langchain-community langchain_experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 5.5 MB/s eta 0:00:00


In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from dotenv import load_dotenv, find_dotenv

In [9]:
from langchain.schema import Document
from langchain.vectorstores import Chroma

In [10]:
from langchain.schema import Document

# Create LangChain documents for IPL players

doc1 = Document(
        page_content="Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.",
        metadata={"team": "Royal Challengers Bangalore"}
    )
doc2 = Document(
        page_content="Rohit Sharma is the most successful captain in IPL history, leading Mumbai Indians to five titles. He's known for his calm demeanor and ability to play big innings under pressure.",
        metadata={"team": "Mumbai Indians"}
    )
doc3 = Document(
        page_content="MS Dhoni, famously known as Captain Cool, has led Chennai Super Kings to multiple IPL titles. His finishing skills, wicketkeeping, and leadership are legendary.",
        metadata={"team": "Chennai Super Kings"}
    )
doc4 = Document(
        page_content="Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.",
        metadata={"team": "Mumbai Indians"}
    )
doc5 = Document(
        page_content="Ravindra Jadeja is a dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super Kings, his quick fielding and match-winning performances make him a key player.",
        metadata={"team": "Chennai Super Kings"}
    )


In [11]:
docs = [doc1, doc2, doc3, doc4, doc5]

In [15]:
vector_store = Chroma(
    embedding_function = HuggingFaceBgeEmbeddings(),
    persist_directory = 'my_chroma_db',
    collection_name = 'sample'
)

/tmp/ipython-input-15-3885962273.py:2: LangChainDeprecationWarning: Default values for HuggingFaceBgeEmbeddings.model_name were deprecated in LangChain 0.2.5 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceBgeEmbeddings constructor instead.
  embedding_function = HuggingFaceBgeEmbeddings(),


In [16]:
# add documents
vector_store.add_documents(docs)

['434b21d2-f0a6-463c-abec-eb31cf461479',
 '1ebf2fb2-9cc1-4b9a-8b03-a6e5b3f4dfa5',
 'b4e6cd5e-632b-4e26-a4dc-4258fc3b358b',
 'f54ecbc9-d459-4e8c-af7d-4fa0ae0de539',
 '6d9b9843-6c11-4631-9860-9d2c8af07513']

In [18]:
# view documents
vector_store.get(include=['embeddings','documents','metadatas'])

{'ids': ['434b21d2-f0a6-463c-abec-eb31cf461479',
  '1ebf2fb2-9cc1-4b9a-8b03-a6e5b3f4dfa5',
  'b4e6cd5e-632b-4e26-a4dc-4258fc3b358b',
  'f54ecbc9-d459-4e8c-af7d-4fa0ae0de539',
  '6d9b9843-6c11-4631-9860-9d2c8af07513'],
 'embeddings': array([[ 0.00285829,  0.00073696, -0.02261552, ..., -0.01859732,
         -0.01296664, -0.02528509],
        [ 0.01100515, -0.00614104, -0.02306416, ..., -0.0270461 ,
         -0.02092154, -0.01513457],
        [ 0.01678874, -0.01916563, -0.00363477, ..., -0.0352846 ,
         -0.00068651, -0.02408634],
        [ 0.0112136 , -0.02619686, -0.04694555, ..., -0.0191632 ,
         -0.03784388, -0.02662324],
        [ 0.01199403, -0.00839731, -0.04070244, ..., -0.02149359,
         -0.01666965, -0.00976742]]),
 'documents': ['Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.',
  "Rohit Sharma is the most successful ca

In [19]:
# Search doucments
vector_store.similarity_search(
    query = 'Who among these are a bowler?',
    k=2
)

[Document(metadata={'team': 'Mumbai Indians'}, page_content='Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.'),
 Document(metadata={'team': 'Chennai Super Kings'}, page_content='Ravindra Jadeja is a dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super Kings, his quick fielding and match-winning performances make him a key player.')]

In [20]:
# Search with similarity score
vector_store.similarity_search_with_score(
    query="",
    filter={'team':"Chennai Super Kings"}
)

[(Document(metadata={'team': 'Chennai Super Kings'}, page_content='MS Dhoni, famously known as Captain Cool, has led Chennai Super Kings to multiple IPL titles. His finishing skills, wicketkeeping, and leadership are legendary.'),
  0.48123759031295776),
 (Document(metadata={'team': 'Chennai Super Kings'}, page_content='Ravindra Jadeja is a dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super Kings, his quick fielding and match-winning performances make him a key player.'),
  0.5538701415061951)]

In [21]:
updated_doc1 = Document(
    page_content="Virat Kohli, the former captain of Royal Challengers Bangalore (RCB), is renowned for his aggressive leadership and consistent batting performances. He holds the record for the most runs in IPL history, including multiple centuries in a single season. Despite RCB not winning an IPL title under his captaincy, Kohli's passion and fitness set a benchmark for the league. His ability to chase targets and anchor innings has made him one of the most dependable players in T20 cricket.",
    metadata={"team": "Royal Challengers Bangalore"}
)

vector_store.update_document(document_id='09a39dc6-3ba6-4ea7-927e-fdda591da5e4', document=updated_doc1)


In [22]:
# view documents
vector_store.get(include=['embeddings','documents', 'metadatas'])

{'ids': ['434b21d2-f0a6-463c-abec-eb31cf461479',
  '1ebf2fb2-9cc1-4b9a-8b03-a6e5b3f4dfa5',
  'b4e6cd5e-632b-4e26-a4dc-4258fc3b358b',
  'f54ecbc9-d459-4e8c-af7d-4fa0ae0de539',
  '6d9b9843-6c11-4631-9860-9d2c8af07513'],
 'embeddings': array([[ 0.00285829,  0.00073696, -0.02261552, ..., -0.01859732,
         -0.01296664, -0.02528509],
        [ 0.01100515, -0.00614104, -0.02306416, ..., -0.0270461 ,
         -0.02092154, -0.01513457],
        [ 0.01678874, -0.01916563, -0.00363477, ..., -0.0352846 ,
         -0.00068651, -0.02408634],
        [ 0.0112136 , -0.02619686, -0.04694555, ..., -0.0191632 ,
         -0.03784388, -0.02662324],
        [ 0.01199403, -0.00839731, -0.04070244, ..., -0.02149359,
         -0.01666965, -0.00976742]]),
 'documents': ['Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.',
  "Rohit Sharma is the most successful ca

In [23]:
# delete document
vector_store.delete(ids=['09a39dc6-3ba6-4ea7-927e-fdda591da5e4'])

In [24]:
# view documents
vector_store.get(include=['embeddings','documents', 'metadatas'])

{'ids': ['434b21d2-f0a6-463c-abec-eb31cf461479',
  '1ebf2fb2-9cc1-4b9a-8b03-a6e5b3f4dfa5',
  'b4e6cd5e-632b-4e26-a4dc-4258fc3b358b',
  'f54ecbc9-d459-4e8c-af7d-4fa0ae0de539',
  '6d9b9843-6c11-4631-9860-9d2c8af07513'],
 'embeddings': array([[ 0.00285829,  0.00073696, -0.02261552, ..., -0.01859732,
         -0.01296664, -0.02528509],
        [ 0.01100515, -0.00614104, -0.02306416, ..., -0.0270461 ,
         -0.02092154, -0.01513457],
        [ 0.01678874, -0.01916563, -0.00363477, ..., -0.0352846 ,
         -0.00068651, -0.02408634],
        [ 0.0112136 , -0.02619686, -0.04694555, ..., -0.0191632 ,
         -0.03784388, -0.02662324],
        [ 0.01199403, -0.00839731, -0.04070244, ..., -0.02149359,
         -0.01666965, -0.00976742]]),
 'documents': ['Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.',
  "Rohit Sharma is the most successful ca